In [1]:
import pandas as pd 
import numpy as np 
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix, make_scorer
from sklearn.metrics import accuracy_score, f1_score, roc_curve, auc, roc_auc_score
from sklearn.model_selection import RepeatedKFold, KFold, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import xgboost
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.pipeline import Pipeline

In [2]:
# LEER LOS DATOS 
file_in = pd.read_csv('dead_due_to_AHE.csv') #lee archivo 
file_in = file_in.drop(['subject_id', 'hadm_id', 'icustay_id'], axis= 'columns') #elimino las columnas que no ocupo 

# IMPUTACIÓN DE DATOS 
x = file_in.drop(['dead'], axis='columns') #para mis variables de entrada soalamente elimino la varaible a predecir
y= file_in.dead #me quedo con la variable a predecir en y 
columnas= x.columns 


# DIVIDIR CONJUNTO DE ENTRENAMIENTO, VALIDACIÓN Y PRUEBA 
x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.3, )
print('X_train:', x_train.shape, 'X_test:', x_test.shape) 

#data = x.replace(" ", np.NaN)
#values = data.values
#imp = IterativeImputer(max_iter=10, random_state=0)
imp = KNNImputer(n_neighbors=5, weights= 'uniform').fit(x_train)
##imp = MissForest()

#imp= SimpleImputer(missing_values=np.nan, strategy='constant').fit(x_train) #mean, most frequent, constant 
x_train = imp.fit_transform(x_train)
x_test = imp.fit_transform(x_test)
print('X_train:', x_train.shape, 'X_test:', x_test.shape)

X_train: (3563, 40) X_test: (1528, 40)
X_train: (3563, 40) X_test: (1528, 40)


In [3]:
print(type(x_train))
x_train_imputado = pd.DataFrame(x_train, columns=[columnas]) #convirtiendo el arreglo en dataframe 
x_test_imputado = pd.DataFrame(x_test, columns=[columnas])
x_train_imputado

<class 'numpy.ndarray'>


,age,kg,creatinine,sexo,first_sicu,first_micu,first_ccu,first_tsicu,first_csru,first_nicu,...,resprate_max,tempc_min,tempc_mean,tempc_max,spo2_min,spo2_mean,spo2_max,pao2_min,pao2_mean,pao2_max
0,66.0,77.000000,2.4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,31.0,35.444446,36.531746,37.444445,84.0,97.093750,100.0,186.0,186.000000,186.000000
1,54.0,134.800003,0.8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,26.0,36.388889,36.881945,37.388891,83.0,93.000000,100.0,63.0,76.500000,90.000000
2,81.0,40.099998,0.6,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,41.0,35.555556,36.611111,38.222224,23.0,93.759358,100.0,63.0,108.090909,361.000000
3,59.0,94.500000,1.1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,40.0,36.222221,37.390909,38.833334,86.0,95.926667,100.0,57.0,116.956522,251.000000
4,50.0,99.500000,0.7,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,27.0,36.166666,36.922997,37.666668,91.0,97.224490,100.0,64.0,196.000000,411.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3558,60.0,75.000000,0.7,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,23.0,35.000000,37.501587,38.599998,94.0,98.971429,100.0,88.0,244.888889,438.000000
3559,85.0,74.000000,1.2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,29.0,35.000000,36.214815,37.200001,91.0,98.702479,100.0,36.0,282.181818,439.000000
3560,52.0,94.800003,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,28.0,34.900002,37.329739,38.900002,82.0,96.677966,100.0,70.0,202.909091,398.000000
3561,63.0,50.000000,0.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,37.0,35.000000,36.062500,37.444445,84.0,95.833333,100.0,91.0,103.200000,137.000000


In [4]:
# ESTANDARIZACIÓN DE DATOS 
estandarizar = ['age', 'kg', 'creatinine','heartrate_min', 'heartrate_mean', 'heartrate_max', 'sysbp_min',
       'sysbp_mean', 'sysbp_max', 'diasbp_min', 'diasbp_mean', 'diasbp_max',
       'meanbp_min', 'meanbp_mean', 'meanbp_max', 'resprate_min','resprate_mean', 'resprate_max', 'tempc_min', 'tempc_mean', 'tempc_max',
       'spo2_min', 'spo2_mean', 'spo2_max', 'pao2_min', 'pao2_mean','pao2_max']
dummies = ['sexo', 'first_sicu', 'first_micu',
       'first_ccu', 'first_tsicu', 'first_csru', 'first_nicu', 'last_sicu','last_micu', 'last_ccu', 'last_tsicu', 'last_csru', 'last_nicu']

#Intentar con las varaibles separadas y también juntas 
scaler = StandardScaler().fit(x_train_imputado)
#transformed_values[estandarizar] = scaler.fit_transform(transformed_values[estandarizar]) #estandarizo las variables númericas 
x_train_escalado = scaler.fit_transform(x_train_imputado)
x_test_escalado = scaler.fit_transform(x_test_imputado)

#COLOCAR PRINTS 
print('X_train:', x_train_escalado.shape, 'X_test:', x_test_escalado.shape)

X_train: (3563, 40) X_test: (1528, 40)


In [5]:
# DEFINIR MODELO A UTILIZAR 
#RF
modeloXGB = xgb.XGBClassifier(use_label_encoder=False)
modeloRF = RandomForestClassifier()
modeloSVM = svm.LinearSVC()
modeloADA = AdaBoostClassifier()
#DEFINIR PARÁMETROS A ITERAR

parametrosRF = {'n_estimators': range(10, 200, 10), 'criterion':['gini', 'entropy'], 'max_depth': range(1,10,1), 
              'max_features':['auto', 'sqrt', 'log2'],  'min_impurity_decrease':[1e-4, 1e-7], 
              'class_weight':['balanced', 'balanced_subsample']} # PARA RF

parametrosRFprueba = {'n_estimators': range(10, 200, 10),'max_depth': range(1,10,1),'class_weight':['balanced']}

ratio = float(np.sum(y_train == 0)) / np.sum(y_train==1)
                      
parametrosXGB = {'loss':['deviance', 'exponential'], 'criterion':['friedman_mse', 'mse', 'mae'], 'max_features':['auto', 'sqrt',
              'log2'], 'n_estimators': range(10, 200, 10), "learning_rate": [0.1,0.05],"colsample_bytree" : [0.4, 0.8],
                    "subsample" : [0.8, 0.4], "reg_alpha" : [0.5], "reg_lambda": [2],
                    "objective": ['binary:logistic'],  'max_depth': range(1,10,1), "gamma":[10],"rate_drop": [0.5, 0.3],
                    "seed": [422], "eval_metric": ['auc'],"scale_pos_weight": [ratio]}

parametrosXGBprueba = { 'n_estimators': range(10, 200, 10), "learning_rate": [0.1,0.05], 'max_depth': range(1,10,1),
                        "eval_metric": ['auc'],"scale_pos_weight": [ratio]}

parametrosSVM = {"C":np.logspace(-3,3,7), "class_weight": ['balanced'],"random_state": [422]}
parametrosSVMprueba = {'C':np.logspace(-3,3,7), 'loss':['hinge', 'squared_hinge'], 'multi_class':['ovr', 'crammer_singer']}

parametrosADA = {'n_estimators': range(10, 200, 10),  "learning_rate": [0.1,0.05], 'algorithm':['SAMME', 'SAMME.R']}

#ENTRENAR GRID SEARCH 
#TIPOS DE SCORE 
balanced = class_weight.compute_sample_weight('balanced', y_train)
RKF = RepeatedKFold(n_splits=5, n_repeats=2, random_state=422) #probar los CROSS VALIDATION
SKF = StratifiedKFold(n_splits=5, shuffle= True, random_state=422)
grid = GridSearchCV(estimator= modeloADA, param_grid= parametrosADA, cv=RKF, scoring='roc_auc', verbose=2, refit= True)

In [6]:
grid.fit(x_train, y_train, sample_weight = balanced)

Fitting 10 folds for each of 76 candidates, totalling 760 fits
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=10; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=10; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=10; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=10; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=10; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=10; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=10; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=10; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=10; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=10; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=20; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.1, n

[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=110; total time=   0.5s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=110; total time=   0.5s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=110; total time=   0.5s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=110; total time=   0.5s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=110; total time=   0.5s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=110; total time=   0.5s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=110; total time=   0.5s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=110; total time=   0.5s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=110; total time=   0.5s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=120; total time=   0.5s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=120; total time=   0.5s
[CV] END algorithm=SAMME, learning_rate=0.1, n_estimators=120; total time=   0.5s
[CV] END algorit

[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=20; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=20; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=20; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=20; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=20; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=20; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=20; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=20; total time=   0.0s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=30; total time=   0.1s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=30; total time=   0.1s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=30; total time=   0.1s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=30; total time=   0.1s
[CV] END algorit

[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=120; total time=   0.5s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=120; total time=   0.5s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=120; total time=   0.5s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=120; total time=   0.5s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=120; total time=   0.5s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=120; total time=   0.5s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=120; total time=   0.5s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=120; total time=   0.5s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=130; total time=   0.6s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=130; total time=   0.6s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=130; total time=   0.6s
[CV] END algorithm=SAMME, learning_rate=0.05, n_estimators=130; total time=   0.6s
[CV]

[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=30; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=30; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=30; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=30; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=30; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=30; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=30; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=30; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=40; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=40; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=40; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=40; total time=   0.1s
[CV]

[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=130; total time=   0.6s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=130; total time=   0.6s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=130; total time=   0.6s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=130; total time=   0.6s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=130; total time=   0.6s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=130; total time=   0.6s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=130; total time=   0.6s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=130; total time=   0.6s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=130; total time=   0.6s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=140; total time=   0.7s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=140; total time=   0.6s
[CV] END algorithm=SAMME.R, learning_rate=0.1, n_estimators=140; total time=

[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=40; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=40; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=40; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=40; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=40; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=40; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=40; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=40; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=40; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=40; total time=   0.1s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=50; total time=   0.2s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=50; total time=

[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=130; total time=   0.6s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=130; total time=   0.6s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=140; total time=   0.6s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=140; total time=   0.6s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=140; total time=   0.6s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=140; total time=   0.6s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=140; total time=   0.7s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=140; total time=   0.7s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=140; total time=   0.6s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=140; total time=   0.6s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=140; total time=   0.7s
[CV] END algorithm=SAMME.R, learning_rate=0.05, n_estimators=140;

GridSearchCV(cv=RepeatedKFold(n_repeats=2, n_splits=5, random_state=422),
             estimator=AdaBoostClassifier(),
             param_grid={'algorithm': ['SAMME', 'SAMME.R'],
                         'learning_rate': [0.1, 0.05],
                         'n_estimators': range(10, 200, 10)},
             scoring='roc_auc', verbose=2)

In [7]:
# METRICAS DEL MODELO 
auc_train = grid.best_score_
std_train = grid.cv_results_['std_test_score'][grid.best_index_]
best_params = grid.best_params_
best_estimator = grid.best_estimator_
print("auc:",auc_train)
print("std_validation:",std_train)
print('best parameters:',best_params)
print('best estimator:', best_estimator)

auc: 0.8384211524646104
std_validation: 0.009902929218743325
best parameters: {'algorithm': 'SAMME.R', 'learning_rate': 0.1, 'n_estimators': 170}
best estimator: AdaBoostClassifier(learning_rate=0.1, n_estimators=170)
